In [ ]:
from checkpoint import openpmdcopy, print_attributes

import math
import matplotlib.pyplot as plt
import numpy as np
import openpmd_api as io
import scipy.constants
from scipy.ndimage import gaussian_filter
import time


## Things to do in the future

* specify a cutoff, behind all fields are written as 0 instead of a small value e.g. 1e-10 * field_max
* multi GPU / multi Node calculations  field size are limited by GPU memory size

## Things to note
* The Convolutional PML fields aren't changed, which might lead to unexpected simulations
* Only use this initialization on the 0 timestep (for now)

## Set path to checkpoint

In [ ]:

# you can use one checkpoint explicitly checkpoint_0.h5
# or a time series with checkpoint_%T.h5
# when giving an explicit checkpoint you can ignore the warning

read_file = "/bigdata/hplsim/scratch/spreng88/runs/poisson/nonR_restart2/simOutput/checkpoints/checkpoint_%T.bp"
output_file = "/bigdata/hplsim/scratch/spreng88/runs/write_%T.bp"

files = openpmdcopy(read_file, output_file)

## General setup

In [ ]:

# add field to iteration 0
input_iteration = files.read_series.iterations[0] 
output_iteration = files.write_series.iterations[0]

# Read attributes of the simulation that might be needed later
# Everything is calculated in PIConGPU units
# For SI units: length_si = value_pic * unit_length
cell_depth = input_iteration.get_attribute("cell_depth")          # z
cell_height = input_iteration.get_attribute("cell_height")        # y
cell_width = input_iteration.get_attribute("cell_width")          # x

unit_efield = input_iteration.get_attribute("unit_efield")
unit_bfield = input_iteration.get_attribute("unit_bfield")
unit_charge = input_iteration.get_attribute("unit_charge")
unit_mass = input_iteration.get_attribute("unit_mass")
unit_speed = input_iteration.get_attribute("unit_speed")
unit_length = input_iteration.get_attribute("unit_length")
unit_time = input_iteration.get_attribute("unit_time")

pi = scipy.constants.pi
c = scipy.constants.c / unit_speed
eps0 = input_iteration.get_attribute("eps0")
mue0 = input_iteration.get_attribute("mue0")

In [ ]:
files.copy_series_data()

In [ ]:
def NGP_assignment(x):
    # possibly wrong if the particle position is = 0.5
    return 1 * (np.abs(x)<=1/2)

def CIC_assignment(x):
    return (1-np.abs(x)) * (np.abs(x)<1)

def TSC_assignment(x):
    return ((3/4 - x**2) * (np.abs(x) < 1/2)
            + 1/2*(3/2 - np.abs(x))**2 * ((np.abs(x)<3/2) & (np.abs(x)>=1/2)))

def PQS_assignment(x):
    return (1/6* (4 - 6 * x**2 + 3 * np.abs(x)**3) * (np.abs(x)<=1)
            + 1/6 * (2-np.abs(x))**3 * ((np.abs(x)<2) & (np.abs(x)>1)))

def PCS_assignment(x):
    return ((115/192 + x**2 * (-5/8 + 1/4 * x**2)) * (np.abs(x) < 1/2)
            + 1/96 * (55 + 4 * np.abs(x) * (5 - 2 * np.abs(x) * (15 + 2 * np.abs(x) * (-5 + np.abs(x))))) * ((np.abs(x) < 3/2) & (np.abs(x)>=1/2))
            + 1/384 * (5 - 2*np.abs(x))**4  * ((np.abs(x)<5/2) & (np.abs(x)>=3/2)) ) 

## load particle data from the checkpoint
define the species identifier string, as in the simulation

In [ ]:
species = "e"

xpos_incell = input_iteration.particles[species]["position"]["x"][:]
ypos_incell = input_iteration.particles[species]["position"]["y"][:]
zpos_incell = input_iteration.particles[species]["position"]["z"][:]
xpos_offset = input_iteration.particles[species]["positionOffset"]["x"][:]
ypos_offset = input_iteration.particles[species]["positionOffset"]["y"][:]
zpos_offset = input_iteration.particles[species]["positionOffset"]["z"][:]
momentumx = input_iteration.particles[species]["momentum"]["x"][:]
momentumy = input_iteration.particles[species]["momentum"]["y"][:]
momentumz = input_iteration.particles[species]["momentum"]["z"][:]
weightings = input_iteration.particles[species]["weighting"][io.Record_Component.SCALAR][:]
charge = input_iteration.particles[species]["charge"][io.Record_Component.SCALAR][:]
mass = input_iteration.particles[species]["mass"][io.Record_Component.SCALAR][:]

files.read_series.flush()

xpos = xpos_incell + np.float32(xpos_offset)
ypos = ypos_incell + np.float32(ypos_offset)
zpos = zpos_incell + np.float32(zpos_offset)

# free some memory
del xpos_incell, ypos_incell, zpos_incell
xpos_offset, ypos_offset, zpos_offset

particleCount = len(mass)

## output to check particleCount and gridsize

In [ ]:
xdim = 32 + 1
ydim = 64 + 1
zdim = 32 + 1
shape = zdim * ydim * xdim
field_shape = (zdim-1) * (ydim-1) * (xdim-1)

In [ ]:
shape_assignment_function = PCS_assignment
#n = 5
##particleCount = 1
#_xpos = []
#_ypos = []
#_zpos = []
#_charge = []
#_momentumx = []
#_momentumy = []
#_momentumz = []
#_weighting = []
#_mass = []
#
def apply_shape_particle():
    for pos_id in range(particleCount):
        x = xpos[pos_id]
        y = ypos[pos_id]
        z = zpos[pos_id]
        #print(x, y ,z)
        dim = np.arange(n)
        charge_density = 0 # np.zeros(shape=(n, n, n))
        
        x_off = xpos_offset[pos_id]
        y_off = ypos_offset[pos_id]
        z_off = zpos_offset[pos_id]
        counter = 0
        for zz in dim:
            for yy in dim:
                for xx in dim:
                    index = xx + yy*n + zz*n*n
                    charge_density = (shape_assignment_function(xx - x + x_off - 2) *
                                      shape_assignment_function(yy - y + y_off - 2) *
                                      shape_assignment_function(zz - z + z_off - 2) *
                                      charge[pos_id])
                    #print(shape_assignment_function(xx - x + x_off - 2),
                    #      shape_assignment_function(yy - y + y_off - 2),
                    #      shape_assignment_function(zz - z + y_off - 2))
                    
                    if charge_density != 0:
                        #print(charge_density, charge_density != 0)
                        _xpos.append(x_off + xx - 2)
                        _ypos.append(y_off + yy - 2)
                        _zpos.append(z_off + zz - 2)
                        _momentumx.append(momentumx[pos_id])
                        _momentumy.append(momentumy[pos_id])
                        _momentumz.append(momentumz[pos_id])
                        _weighting.append(weightings[pos_id])
                        _mass.append(mass[pos_id])
                        _charge.append(charge_density)
                        
                        
charge_density_global = np.zeros(shape=(zdim, ydim, xdim))

def apply_shape_density():
    n = 5
    for pos_id in range(particleCount):
        x = xpos[pos_id]
        y = ypos[pos_id]
        z = zpos[pos_id]
        
        dim = np.arange(n)
        charge_density = np.zeros(shape=(n, n, n))
        
        x_off = xpos_offset[pos_id]
        y_off = ypos_offset[pos_id]
        z_off = zpos_offset[pos_id]
        counter = 0
        for zz in dim:
            for yy in dim:
                for xx in dim:
                    index = xx + yy*n + zz*n*n
                    charge_density[zz, yy, xx] = (shape_assignment_function(xx - x + x_off - 2.0) *
                                                  shape_assignment_function(yy - y + y_off - 2.0) *
                                                  shape_assignment_function(zz - z + z_off - 2.0) *
                                                  charge[pos_id] / (cell_width * cell_height * cell_depth))
                    #print(shape_assignment_function(xx - x + x_off - 2),
                    #      shape_assignment_function(yy - y + y_off - 2),
                    #      shape_assignment_function(zz - z + y_off - 2))
                    #print(xx - x + x_off - 2,yy - y + y_off - 2,zz - z + z_off - 2)

                    
        print("sum:", np.sum(charge_density))
        print("charge:", charge[pos_id])
        print(charge_density)
        charge_density_global[z_off-2:z_off+3, y_off-2:y_off+3, x_off-2:x_off+3] = charge_density
                        
apply_shape_density()

#xpos = _xpos
#ypos = _ypos
#zpos = _zpos
#charge = _charge
#momentumx = _momentumx
#momentumy = _momentumy
#momentumz = _momentumz
#weighting = _weighting
#mass = _mass
#density_flat = np.zeros(shape=(zdim, ydim, xdim))
#density_flat[15, 15, 15] = -charge[0]/eps0
density_flat = -charge_density_global.reshape((zdim) * (ydim) * (xdim)) / eps0

In [ ]:
density_flat.sum()

In [ ]:
print("particleCount:", particleCount, len(mass))
input_field_shape = files.input_iteration.meshes["E"]["x"].shape
xdim_total = input_field_shape[2]
ydim_total = input_field_shape[1]
zdim_total = input_field_shape[0]
input_field_shape_total = xdim_total * ydim_total * zdim_total
print("gridSize (z,y,x, total):", input_field_shape, input_field_shape_total)

field_dtype = files.input_iteration.meshes['E']['x'].dtype
field_dataset = io.Dataset( field_dtype, input_field_shape)

for field_name in ['E', 'B']:
    files.copy_attributes(files.input_iteration.meshes[field_name],
                          files.output_iteration.meshes[field_name])
    for component in ['x', 'y', 'z']:
        files.copy_attributes(files.input_iteration.meshes[field_name][component],
                              files.output_iteration.meshes[field_name][component])
        files.output_iteration.meshes[field_name][component].reset_dataset(field_dataset)

In [ ]:
charge

## calculate the fields

set the chunk_size to the number of cells if the gpu has enough memory

In [ ]:
#
#chunk_size = np.array([768, 512, 768], dtype=np.int32)  # z y x
#xdim = chunk_size[2]
#ydim = chunk_size[1]
#zdim = chunk_size[0]
#shape = xdim * ydim * zdim
#
#min_memory_device = shape * 24 + particleCount * 56
#print("device needs a minimum memory of:",min_memory_device, "Bytes = {:.5} GB".format(min_memory_device*1e-9))
#
#chunk_offset = np.array([0, 128, 0], dtype=np.int32)          # offset for chunk (z, y, z)
#

In [ ]:
pot = np.zeros(((xdim)*(ydim)*(zdim))**2).reshape((xdim)*(ydim)*(zdim), (xdim)*(ydim)*(zdim))

for x in range(xdim):
    for y in range(ydim):
        for z in range(zdim):
            index = x + y * xdim + z * xdim * ydim
            #print(x, y, z)
            
            pot[ x + y * xdim + z * xdim * ydim, index] = -2 * ( 1/cell_width**2 + 1/cell_height**2 + 1/cell_depth**2 )
            #print(x + (y+1) * xdim + z * xdim * ydim, index)
            #print(x + (y-1) * xdim + z * xdim * ydim, index)
            
            if x+1 < xdim:
                pot[ (x+1) + y * xdim + z * xdim * ydim, index] = 1 / (cell_width**2)
            if x-1 > -1:
                pot[ (x-1) + y * xdim + z * xdim * ydim, index] = 1 / (cell_width**2)
            
            if y+1 < ydim:
                pot[ x + (y+1) * xdim + z * xdim * ydim, index] = 1 / (cell_height**2)
            if y-1 > -1:
                pot[ x + (y-1) * xdim + z * xdim * ydim, index] = 1 / (cell_height**2)
            
            if z+1 < zdim:
                pot[ x + y * xdim + (z+1) * xdim * ydim, index] = 1 / (cell_depth**2)
            if z-1 > -1:
                pot[ x + y * xdim + (z-1) * xdim * ydim, index] = 1 / (cell_depth**2)

In [ ]:
S = scipy.sparse.csr_matrix(-pot)
del pot

In [ ]:

array_dtype = np.float32                            # dtype for arrays
ex = np.zeros(field_shape, dtype=array_dtype)
ey = np.zeros(field_shape, dtype=array_dtype)
ez = np.zeros(field_shape, dtype=array_dtype)
bx = np.zeros(field_shape, dtype=array_dtype)
by = np.zeros(field_shape, dtype=array_dtype)
bz = np.zeros(field_shape, dtype=array_dtype)


print("\nstart of field calculation time:", time.ctime())
starttime = time.time()

solution = np.array(scipy.sparse.linalg.cg(S, density_flat, tol=1e-12)[0])


exeTime = time.time()-starttime
print("time: {:.5}".format( exeTime ), "s")
print("avgTime per particle per cell:", exeTime / particleCount / (xdim*ydim*zdim), "s/(particle*cell)")


In [ ]:
solution = solution.reshape(zdim, ydim, xdim)

ex = np.float32((solution[:-1,:-1,1:] - solution[:-1,:-1,:-1])/cell_width) 
ey = np.float32((solution[:-1,1:,:-1] - solution[:-1,:-1,:-1])/cell_height) 
ez = np.float32((solution[1:,:-1,:-1] - solution[:-1,:-1,:-1])/cell_depth) 

In [ ]:
chunk_offset=[0,0,0]
chunk_size = [zdim-1, ydim-1, xdim-1]
# write the calculate chunk of the field 
output_iteration.meshes['E']['x'].store_chunk(ex.reshape(xdim-1, ydim-1, zdim-1))#, chunk_offset, chunk_size)
output_iteration.meshes['E']['y'].store_chunk(ey.reshape(xdim-1, ydim-1, zdim-1))#, chunk_offset, chunk_size)
output_iteration.meshes['E']['z'].store_chunk(ez.reshape(xdim-1, ydim-1, zdim-1))#, chunk_offset, chunk_size)
output_iteration.meshes['B']['x'].store_chunk(bx.reshape(xdim-1, ydim-1, zdim-1))#, chunk_offset, chunk_size)
output_iteration.meshes['B']['y'].store_chunk(by.reshape(xdim-1, ydim-1, zdim-1))#, chunk_offset, chunk_size)
output_iteration.meshes['B']['z'].store_chunk(bz.reshape(xdim-1, ydim-1, zdim-1))#, chunk_offset, chunk_size)
files.write_series.flush()

print('done')

## When finished with all field chunks
you can write the other data to the file, except E, B fields that we wrote with this program

In [ ]:
del ex, ey, ez
del bx, by, bz

In [ ]:
# free memory for copy
del charge
del xpos
del ypos
del zpos
del momentumx
del momentumy
del momentumz
del mass
del weighting

In [ ]:
files.copy(exclude_mesh=['E', 'B'])

# Visualization of the fields if necessary

and other stuff, not realy documented with comments

## absolute field values

In [ ]:
# absolute values of the fields
ef = np.sqrt(ex**2+ey**2+ez**2)
bf = np.sqrt(bx**2+by**2+bz**2)

In [ ]:
plt.figure(figsize=(15,15))
depth = 12
plt.imshow(((ex.reshape(zdim-1, ydim-1, xdim-1)[depth,:,:].T)))
#plt.xlim(290, 320)
#plt.ylim(160,200)
plt.colorbar()
plt.show()

In [ ]:
print(xpos, ypos, zpos)

## show particle positions

In [ ]:
plt.figure(figsize=(15,15))
a = np.histogram2d(xpos, ypos, weights=weightings, bins=[np.linspace(0, xdim, xdim+1), np.linspace(0, ydim, ydim+1)] )
plt.imshow(a[0])
plt.xlabel("y cells")
plt.ylabel("x cells")
plt.show()

In [ ]:
del ex_d
del ey_d
del ez_d
del bx_d
del by_d
del bz_d

In [ ]:
del writeSeries

## Further calculations for the generated data
the expected values are

\begin{equation}
\vec{\nabla} \cdot \vec{E} = \frac{\rho(\vec{r})}{\epsilon_0}
\end{equation}

\begin{equation}
\vec{\nabla} \times \vec{E} = 0
\end{equation}

\begin{equation}
\vec{\nabla} \cdot \vec{B} = 0
\end{equation}


In [ ]:
ex = ex.reshape(zdim-1, ydim-1, xdim-1)
ey = ey.reshape(zdim-1, ydim-1, xdim-1)
ez = ez.reshape(zdim-1, ydim-1, xdim-1)

bx = bx.reshape(zdim-1, ydim-1, xdim-1)
by = by.reshape(zdim-1, ydim-1, xdim-1)
bz = bz.reshape(zdim-1, ydim-1, xdim-1)

divE = ((ex[1:, 1:, 1:] - ex[1:, 1:, :-1]) / cell_width +
           (ey[1:, 1:, 1:] - ey[1:, :-1, 1:]) / cell_height +
           (ez[1:, 1:, 1:] - ez[:-1, 1:, 1:]) / cell_depth)

divB = ((bx[1:, 1:, 1:] - bx[1:, 1:, :-1]) / cell_width +
       (by[1:, 1:, 1:] - by[1:, :-1, 1:]) / cell_height +
       (bz[1:, 1:, 1:] - bz[:-1, 1:, 1:]) / cell_depth) * unit_bfield * 299792458 / unit_length * scipy.constants.epsilon_0


In [ ]:
print(np.sum(divE))
plt.figure(figsize=(20,20))
field = divE[15].T#-charge_density_global[15,1:-1,1:-1].T/eps0
plt.imshow(((field)))
#plt.xlim(200,400)
#plt.ylim(300,450)
plt.colorbar()

In [ ]:
del divE, divB

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(unit_efield*ez[:,:,128])
plt.colorbar()
plt.xlim(120,140)
plt.ylim(120,140)

In [ ]:
xx,yy,zz = 50,50,50    # width height depth - x y z
curlEx = np.zeros_like(ex)
curlEy = np.zeros_like(ex)
curlEz = np.zeros_like(ex)
curlBx = np.zeros_like(ex)
curlBy = np.zeros_like(ex)
curlBz = np.zeros_like(ex)

#for x in np.arange(100, 160):
#    for y in np.arange(100, 160):
#        for z in np.arange(100, 160):
#            curlEx[z,y,x] = (ez[z, y+1, x] - ez[z, y, x]) / cell_height - (ey[z+1, y, x] - ey[z, y, x]) / cell_depth
#            curlEy[z,y,x] = (ex[z+1, y, x] - ex[z, y, x]) / cell_depth  - (ez[z, y, x+1] - ez[z, y, x]) / cell_width
#            curlEz[z,y,x] = (ey[z, y, x+1] - ey[z, y, x]) / cell_width  - (ex[z, y+1, x] - ex[z, y, x]) / cell_height
#
#            curlBx[z,y,x] = (bz[z, y, x] - bz[z, y-1, x]) / cell_height - (by[z, y, x] - by[z-1, y, x]) / cell_depth
#            curlBy[z,y,x] = (bx[z, y, x] - bz[z-1, y, x]) / cell_depth  - (bz[z, y, x] - bz[z, y, x-1]) / cell_width
#            curlBz[z,y,x] = (by[z, y, x] - bz[z, y, x-1]) / cell_width  - (bx[z, y, x] - by[z, y-1, x]) / cell_height


curlEx = (ez[:-1, 1:, :-1] - ez[:-1, :-1, :-1]) / cell_height - (ey[1:, :-1, :-1] - ey[:-1, :-1, :-1]) / cell_depth
curlEy = (ex[1:, :-1, :-1] - ex[:-1, :-1, :-1]) / cell_depth  - (ez[:-1, :-1, 1:] - ez[:-1, :-1, :-1]) / cell_width
curlEz = (ey[:-1, :-1, 1:] - ey[:-1, :-1, :-1]) / cell_width  - (ex[:-1, 1:, :-1] - ex[:-1, :-1, :-1]) / cell_height
curlBx = (bz[1:, 1:, 1:] - bz[1:, :-1, 1:]) / cell_height - (by[1:, 1:, 1:] - by[:-1, 1:, 1:]) / cell_depth
curlBy = (bx[1:, 1:, 1:] - bz[:-1, 1:, 1:]) / cell_depth  - (bz[1:, 1:, 1:] - bz[1:, 1:, :-1]) / cell_width
curlBz = (by[1:, 1:, 1:] - bz[1:, 1:, :-1]) / cell_width  - (bx[1:, 1:, 1:] - by[1:, :-1, 1:]) / cell_height

In [ ]:
plt.figure(figsize=(10,10))
field = curlEz[374,:,:].T
plt.imshow(((field)))
plt.xlim(250,350)
plt.ylim(300,500)
plt.colorbar()